In [72]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import time

import torch
from torchtext import data, datasets, vocab
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from pythainlp.tokenize import word_tokenize

In [3]:
word_file = pd.read_csv("รวมคำ.csv")

In [4]:
word_file.head()

,ดี,1
0,ขอบคุณ,1
1,สู้เสมอ,1
2,ผ่าน,1
3,โคตรสนุก,1
4,สุดยอด,1


In [5]:
word_list = word_file.values.tolist()

print(word_list[0])

['ขอบคุณ', 1]


In [6]:
new_word_list = []

for a_list in word_list:
    new_list = word_tokenize(a_list[0])
    label_list = [new_list, a_list[1]]
    new_word_list.append(label_list)

print(new_word_list)

, [['สาน', 'ฝัน', 'ให้', 'จริง'], 1], [['อาสา'], 1], [['อยาก', 'กอด'], 1], [['เตือนสติ'], 1], [['สันติสุข'], 1], [['สำนึก'], 1], [['เลือก', 'ตามใจ'], 1], [['โค', 'รต', 'น่ารัก'], 1], [['มหัศจรรย์'], 1], [['สิ่ง', 'พิเศษ'], 1], [['วิเศษ'], 1], [['ผ่อนคลาย'], 1], [['ยอมรับ'], 1], [['เจริญรุ่งเรือง'], 1], [['เสียสละ'], 1], [['ไถ่โทษ'], 1], [['ไถ่บาป'], 1], [['อบอวล'], 1], [['สะอาด'], 1], [['เอาใจใส่'], 1], [['ภาวนา'], 1], [['ฟุ้งเฟื่อง'], 1], [['เกียรติคุณ'], 1], [['ด้าน', 'บวก'], 1], [['ปลื้ม', 'ดีใจ'], 1], [['คึกคัก'], 1], [['เจตนา', 'ดี'], 1], [['จิต', 'แจ่มใส'], 1], [['เปรี้ยงปร้าง'], 1], [['ฮิต'], 1], [['ที่', 'นิยม'], 1], [['ปลอบใจ'], 1], [['ปรองดอง'], 1], [['ไมตรี'], 1], [['มารยาท'], 1], [['รอบคอบ'], 1], [['บริสุทธิ์ใจ'], 1], [['เคารพนับถือ'], 1], [['วุฒิภาวะ'], 1], [['สุขุม'], 1], [['ฟื้นฟู'], 1], [['ล้ำสมัย'], 1], [['ดี', 'ที่สุด'], 1], [['ทันเวลา', 'พอดี'], 1], [['สุจริต'], 1], [['น่าเชื่อถือ'], 1], [['เท่ห์'], 1], [['ไว้วางใจ'], 1], [['เสน่ห์'], 1], [['ทรงพลัง'], 1], [['ฮัก'], 

In [7]:
new_word_list[234][0]

['จีบ', 'หวาน']

In [8]:
word_set = set()

for a_list in new_word_list:
    word_set.update(a_list[0])
        
print(word_set)

{'หรอ', 'หลาย', 'เดี๋ยวนี้', 'หิว', 'ประจาน', 'คุย', 'เนร', 'ความสูญเสีย', 'ทอดทิ้ง', 'ทิ้ง', 'ชื่นชม', 'ซัก', 'เตรียมตัว', 'ใส', 'เสมอ', 'เค้า', 'การศึกษา', 'เศรษฐกิจ', 'เเม่ง', 'ดำรง', 'เจริญเติบโต', 'ผู้ชาย', 'ทางใจ', 'เหยียดหยาม', 'เจ็บ', 'ขัดสน', 'อดใจ', 'ต่อเนื่อง', 'เสือก', 'ออกจากงาน', 'อันตรา', 'ทวง', 'ปวดหัว', 'แบบ', 'ดีมาก', 'ศรัทธา', 'ภาวนา', 'ครีม', 'เสียชีวิต', 'แท้', 'หมด', 'ตอแหล', 'สุขสันต์', 'ที่', 'บางครั้ง', 'ลาก่อน', 'เหม็น', 'เกียรติคุณ', 'เสร็จ', 'สิ่ง', 'อย่าง', 'สยอง', 'ปรารถนา', 'ธรรมดา', 'ปิดกั้น', 'ไป', 'เหยียด', 'เหงา', 'หนาว', 'ความปรารถนา', 'เสียน้ำตา', 'ล่วงละเมิด', 'ทันเวลา', 'รูป', 'รอดชีวิต', 'เหะ', 'น่าชื่นชม', 'อาบัติ', 'เจอ', 'ทุกที่', 'พร', 'หรือไง', 'ให้', 'แข็งแกร่ง', 'รู้ตัว', 'ฝัน', 'หยาบคาย', 'ๆ', 'ความเครียด', 'กำ', 'พระ', 'แง่ร้าย', 'คิด', 'สูง', 'จิต', 'ค่ะ', 'สภาวะ', 'สลด', 'ลับหลัง', 'สะอาด', 'เหินห่าง', 'กบฏ', 'เลย', 'ก็', 'เซอร์ไพรส์', 'เนอะ', 'ครื้น', 'กังวล', 'กาก', 'ด่วน', 'ไข้หวัด', 'น่ารำคาญ', 'เวียนหัว', 'ถ่าย', 'ใส่', 'ในหลวง', 

In [9]:
word_set_to_list = list(word_set)

print(word_set_to_list)

['หรอ', 'หลาย', 'เดี๋ยวนี้', 'หิว', 'ประจาน', 'คุย', 'เนร', 'ความสูญเสีย', 'ทอดทิ้ง', 'ทิ้ง', 'ชื่นชม', 'ซัก', 'เตรียมตัว', 'ใส', 'เสมอ', 'เค้า', 'การศึกษา', 'เศรษฐกิจ', 'เเม่ง', 'ดำรง', 'เจริญเติบโต', 'ผู้ชาย', 'ทางใจ', 'เหยียดหยาม', 'เจ็บ', 'ขัดสน', 'อดใจ', 'ต่อเนื่อง', 'เสือก', 'ออกจากงาน', 'อันตรา', 'ทวง', 'ปวดหัว', 'แบบ', 'ดีมาก', 'ศรัทธา', 'ภาวนา', 'ครีม', 'เสียชีวิต', 'แท้', 'หมด', 'ตอแหล', 'สุขสันต์', 'ที่', 'บางครั้ง', 'ลาก่อน', 'เหม็น', 'เกียรติคุณ', 'เสร็จ', 'สิ่ง', 'อย่าง', 'สยอง', 'ปรารถนา', 'ธรรมดา', 'ปิดกั้น', 'ไป', 'เหยียด', 'เหงา', 'หนาว', 'ความปรารถนา', 'เสียน้ำตา', 'ล่วงละเมิด', 'ทันเวลา', 'รูป', 'รอดชีวิต', 'เหะ', 'น่าชื่นชม', 'อาบัติ', 'เจอ', 'ทุกที่', 'พร', 'หรือไง', 'ให้', 'แข็งแกร่ง', 'รู้ตัว', 'ฝัน', 'หยาบคาย', 'ๆ', 'ความเครียด', 'กำ', 'พระ', 'แง่ร้าย', 'คิด', 'สูง', 'จิต', 'ค่ะ', 'สภาวะ', 'สลด', 'ลับหลัง', 'สะอาด', 'เหินห่าง', 'กบฏ', 'เลย', 'ก็', 'เซอร์ไพรส์', 'เนอะ', 'ครื้น', 'กังวล', 'กาก', 'ด่วน', 'ไข้หวัด', 'น่ารำคาญ', 'เวียนหัว', 'ถ่าย', 'ใส่', 'ในหลวง', 

In [10]:
label_set = set()

for a_list in new_word_list:
    if a_list[1] in label_set:
        pass
    else:
        label_set.add(a_list[1])


print(label_set)

{1, 2, 3, 4, 5}


In [11]:
print(len(word_set))
print(len(label_set))

1457
5


In [12]:
sentence_idx = np.linspace(0,len(word_set_to_list),len(word_set_to_list),False)

sentence_idx[0]

0.0

In [15]:
a = 0
link_list_1 = []

while a < len(sentence_idx):
    inner_list = []
    inner_list.append(word_set_to_list[a])
    inner_list.append(sentence_idx[a])
    link_list_1.append(inner_list)
    a += 1

print(link_list_1)

93.0], ['เศร้าสลด', 394.0], ['อุบัติเหตุ', 395.0], ['ลุกโชน', 396.0], ['คอย', 397.0], ['สถานะ', 398.0], ['กาลเวลา', 399.0], ['น่าเชื่อถือ', 400.0], ['เขวี้ยง', 401.0], ['รบกวน', 402.0], ['ไม่', 403.0], ['กระทืบ', 404.0], ['อดีต', 405.0], ['ที่พึ่ง', 406.0], ['เพื่อนสนิท', 407.0], ['คาดหวัง', 408.0], ['ปลื้มใจ', 409.0], ['เป็นกลาง', 410.0], ['ท่วม', 411.0], ['สงคราม', 412.0], ['เสียใจ', 413.0], ['มลทิน', 414.0], ['กำไร', 415.0], ['จรจัด', 416.0], ['เด็ดขาด', 417.0], ['หวานใจ', 418.0], ['หลอก', 419.0], ['หลง', 420.0], ['นาน', 421.0], ['ตั้งใจ', 422.0], ['วิบัติ', 423.0], ['ขาดดุล', 424.0], ['พอ', 425.0], ['เพียง', 426.0], ['คำพูด', 427.0], ['ดัดจริต', 428.0], ['ประกาศ', 429.0], ['ย่อยยับ', 430.0], ['อ่อนแอ', 431.0], ['พัก', 432.0], ['เถอะ', 433.0], ['เย', 434.0], ['ปลดระวาง', 435.0], ['กลั่นแกล้ง', 436.0], ['รวดเร็วทันใจ', 437.0], ['ไอ', 438.0], ['การกระทำ', 439.0], ['ล้ำสมัย', 440.0], ['ไมตรี', 441.0], ['ทราบ', 442.0], ['งง', 443.0], ['แปะ', 444.0], ['ตัดสินใจ', 445.0], ['อยาก', 446.0],

In [43]:
new_dict = dict(link_list_1)

print(new_dict)

ด': 265.0, 'บิน': 266.0, 'สมบูรณ์แบบ': 267.0, 'ล่าช้า': 268.0, 'เท่านั้นเอง': 269.0, 'ธรรม': 270.0, 'แกล้ง': 271.0, 'สิ่งดีงาม': 272.0, 'เท่า': 273.0, 'เอ็นดู': 274.0, 'ละ': 275.0, 'อัปยศ': 276.0, 'คิดถึง': 277.0, 'ทำดี': 278.0, 'มองโลกในแง่ดี': 279.0, 'ชาวบ้าน': 280.0, 'เวิ้งว้าง': 281.0, 'ร่วมมือ': 282.0, 'หนุ่ม': 283.0, 'ไตร่ตรอง': 284.0, 'หยิบยื่น': 285.0, 'ฟุ้งเฟื่อง': 286.0, 'เพราะ': 287.0, 'มุมมอง': 288.0, 'ชั่ว': 289.0, 'ความรู้': 290.0, 'มากขึ้น': 291.0, 'สุดความสามารถ': 292.0, 'การตัดสินใจ': 293.0, 'อีกไม่นาน': 294.0, 'เดิน': 295.0, 'ผิด': 296.0, 'มึง': 297.0, 'สนุกสนาน': 298.0, 'งาน': 299.0, 'สาน': 300.0, 'เจตนา': 301.0, 'เลื่อน': 302.0, 'เคารพนับถือ': 303.0, 'ยืนนาน': 304.0, 'วุ่นวาย': 305.0, 'พนัน': 306.0, 'พลาด': 307.0, 'โสโครก': 308.0, 'ประเสริฐ': 309.0, 'ตื่น': 310.0, 'พิเศษ': 311.0, 'นึกถึง': 312.0, 'วุฒิภาวะ': 313.0, 'ลืม': 314.0, 'เพ': 315.0, 'เอง': 316.0, 'กำลังจะ': 317.0, 'อบอวล': 318.0, 'ทำตัว': 319.0, 'ขัดขวาง': 320.0, 'แล้ง': 321.0, 'นิยาม': 322.0, 'ชาว': 323.0,

In [65]:
new_list_2 = []

for a_list in new_word_list:
    list_of_word_num = []
    for a in a_list[0]:
        b = new_dict[a]
        list_of_word_num.append(b)
    list_of_word_tensor = torch.LongTensor(list_of_word_num)
    list_combine = []
    list_combine.append(a_list[1])
    list_combine.append(list_of_word_tensor)
    new_list_2.append(list_combine)

print(new_list_2)

sor([583])], [2, tensor([ 732, 1206])], [2, tensor([371,  99])], [2, tensor([847, 763])], [2, tensor([847])], [2, tensor([ 371, 1412,   99])], [2, tensor([732])], [2, tensor([371])], [3, tensor([447])], [3, tensor([980])], [3, tensor([14])], [3, tensor([1280])], [3, tensor([458])], [3, tensor([338])], [3, tensor([351])], [3, tensor([845])], [3, tensor([1013])], [3, tensor([581])], [3, tensor([118])], [3, tensor([228])], [3, tensor([1331])], [3, tensor([233])], [3, tensor([1365])], [3, tensor([388])], [3, tensor([143])], [3, tensor([1353])], [3, tensor([1372])], [3, tensor([432])], [3, tensor([1318])], [3, tensor([1249])], [3, tensor([798])], [3, tensor([1285])], [3, tensor([1168])], [3, tensor([323])], [3, tensor([925])], [3, tensor([280])], [3, tensor([1242])], [3, tensor([1090])], [3, tensor([1159])], [3, tensor([996])], [3, tensor([1123])], [3, tensor([970])], [3, tensor([265])], [3, tensor([0])], [3, tensor([1160])], [3, tensor([11])], [3, tensor([566])], [3, tensor([446])], [3, te

In [76]:
train_data, test_data = train_test_split(new_list_2, test_size=0.3, random_state=42)

train_data

[[4, tensor([419,  72, 264, 324])],
 [4, tensor([40, 35])],
 [1, tensor([ 403,  643,  403,  551, 1216, 1382,  407])],
 [1, tensor([187])],
 [3, tensor([143])],
 [4, tensor([833, 811])],
 [1, tensor([ 870, 1109, 1206])],
 [3, tensor([398])],
 [1, tensor([543, 527])],
 [1, tensor([1000, 1019,  397,  746,  723])],
 [4, tensor([615])],
 [1, tensor([729,   9,  15])],
 [4, tensor([  72, 1359,  256,  953])],
 [3, tensor([447])],
 [1, tensor([1037, 1285,  571])],
 [4, tensor([569])],
 [4, tensor([137])],
 [1, tensor([35])],
 [1, tensor([ 731,  846, 1056, 1430,  844,   43,  422,  372,  987])],
 [1, tensor([1443])],
 [4, tensor([259])],
 [3, tensor([299])],
 [1, tensor([  59,  911,  140, 1014, 1137])],
 [4, tensor([ 244, 1082,  131,  327, 1345])],
 [1, tensor([1079,  156, 1159,   43,  475,  145,  349, 1321,  156,  114])],
 [4, tensor([650])],
 [3, tensor([773])],
 [4, tensor([139, 263,  55,  40])],
 [1, tensor([1274])],
 [1, tensor([ 997, 1364])],
 [4, tensor([414])],
 [1, tensor([ 457,  475, 12

In [80]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [81]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
vocab_size = len(word_set)
tagset_size = len(word_set)

In [ ]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [71]:
"""
Functions used to generate batch

Since the text entries have different lengths, a custom function generate_batch() is used to generate data batches and offsets. The function is passed to collate_fn in torch.utils.data.DataLoader. The input to collate_fn is a list of tensors with the size of batch_size, and the collate_fn function packs them into a mini-batch. Pay attention here and make sure that collate_fn is declared as a top level def. This ensures that the function is available in each worker.

The text entries in the original data batch input are packed into a list and concatenated as a single tensor as the input of nn.EmbeddingBag. The offsets is a tensor of delimiters to represent the beginning index of the individual sequence in the text tensor. Label is a tensor saving the labels of individual text entries.
"""

5

In [48]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [59]:
print(training_data[0][0])

print(word_to_ix)

print(tag_to_ix)

['The', 'dog', 'ate', 'the', 'apple']
{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}
{'DET': 0, 'NN': 1, 'V': 2}


In [78]:
prepare_sequence(training_data[0][0],word_to_ix)

tensor([0, 1, 2, 3, 4])

In [73]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [72]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [10]:
VOCAB_SIZE = len(word_set) #เป็นจำนวนคำที่อยู่ใน dataset ของเรา
EMBED_DIM = 32 #จำนวน Dimension ที่เราต้องการจะ Set
NUN_CLASS = len(label_set) #จำนวนชนิดของ Label
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS) #Set Model

In [22]:
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](root='./.data', ngrams=NGRAMS, vocab=None)

120000lines [00:12, 9241.00lines/s]
120000lines [00:21, 5545.47lines/s]
7600lines [00:01, 5660.99lines/s]


In [17]:
VOCAB_SIZE = len(train_dataset.get_vocab())

In [26]:
NUN_CLASS = len(train_dataset.get_labels())

In [16]:
train_dataset[0]

(2,
 tensor([    572,     564,       2,    2326,   49106,     150,      88,       3,
            1143,      14,      32,      15,      32,      16,  443749,       4,
             572,     499,      17,      10,  741769,       7,  468770,       4,
              52,    7019,    1050,     442,       2,   14341,     673,  141447,
          326092,   55044,    7887,     411,    9870,  628642,      43,      44,
             144,     145,  299709,  443750,   51274,     703,   14312,      23,
         1111134,  741770,  411508,  468771,    3779,   86384,  135944,  371666,
            4052]))

In [138]:
train_dataset[125]

(3,
 tensor([    675,   22073,    1023,    7821,    5438,      14,      36,      15,
              36,      16,     675,    6116,      25,      57,    5438,      12,
            1023,    7821,      76,       4,    4388,     289,     557,      11,
               3,     617,    2060,       7,    3314,     957,     281,       4,
             136,      24,   24454,      28,       3,     175,    1061,       9,
             136,     414,     601,    2496,     137,     229,       5,      27,
           19983,      12,       6,    2008,       7,   16442,       2,  641006,
          822127,   16752,   96934,  100213,      62,      63,      71,      70,
            7249,  331557,  371430,    1469,  362271,   49908,   26860,   16752,
          339860,     896,   17248,  895502,    2148,   24332,      79,    7334,
          256655,   13613,  132890,    8989,   15619,   17158,     251,    2067,
          385580, 1095253,     207,    2567,    2463,   48704,   44067,  139821,
          194278,   4145